<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Algorithm-1.2" data-toc-modified-id="Algorithm-1.2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Algorithm 1.2</a></span></li><li><span><a href="#Algorithm-1.3" data-toc-modified-id="Algorithm-1.3-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Algorithm 1.3</a></span><ul class="toc-item"><li><span><a href="#Check-dimensions" data-toc-modified-id="Check-dimensions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Check dimensions</a></span></li><li><span><a href="#Check-values" data-toc-modified-id="Check-values-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Check values</a></span></li><li><span><a href="#Build-Atilde-and-DMD-Modes" data-toc-modified-id="Build-Atilde-and-DMD-Modes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Build Atilde and DMD Modes</a></span></li><li><span><a href="#DMD-Spectra" data-toc-modified-id="DMD-Spectra-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DMD Spectra</a></span></li></ul></li><li><span><a href="#Algorithm-1.4" data-toc-modified-id="Algorithm-1.4-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Algorithm 1.4</a></span></li><li><span><a href="#Algorithm-1.5" data-toc-modified-id="Algorithm-1.5-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Algorithm 1.5</a></span></li></ul></div>

## Algorithm 1.2

In [1]:
import numpy as np

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-white') # use sans-serif fonts

rcParams['figure.figsize'] = 5, 5

label_size = 10
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 15

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0


In [2]:
## Define time and space discretizations

xi = np.linspace(-10, 10, 400)
ti = np.linspace(0, 4*np.pi, 200)
dt = ti[1] - ti[0]
[xgrid, tgrid] = np.meshgrid(xi,ti)


## Create two spatio-temporal patterns
f1 = (1/np.cosh(xgrid + 3))*(1*np.exp(1j*2.3*tgrid))
f2 = ((1/np.cosh(xgrid))*np.tanh(xgrid))*(2*np.exp(1j*2.8*tgrid))

## Combine signals and make data matrix
f = f1 + f2
X = f.T

In [3]:
%matplotlib

alpha_val = 1.0

fig = plt.figure(figsize = (8,8))
ax_f1 = fig.add_subplot(221, projection='3d')
ax_f2 = fig.add_subplot(222, projection='3d')
ax_f = fig.add_subplot(223, projection='3d')

ax_f1.plot_surface(xgrid.T, tgrid.T, np.real(f1.T), \
                   antialiased = False, cmap = 'gray_r', \
                   linewidth = 0, alpha = alpha_val)
# ax_f1.view_init(30,-50)
ax_f1.view_init(40,-120)

ax_f2.plot_surface(xgrid.T, tgrid.T, np.real(f2.T), \
                   antialiased = False, cmap = 'gray_r', \
                   linewidth = 0, alpha = alpha_val)
# ax_f2.view_init(30,-50)
ax_f2.view_init(40,-120)

ax_f.plot_surface(xgrid.T, tgrid.T, np.real(X), \
                  antialiased = False, cmap = 'gray_r', \
                  linewidth = 0, alpha = alpha_val)
# ax_f.view_init(30,-50)
ax_f.view_init(40,-120)

# plt.savefig('test.pdf', bbox_inches = 'tight')
plt.show()



Using matplotlib backend: MacOSX


## Algorithm 1.3

In [4]:
from numpy.linalg import svd, inv, eig

## Create DMD data matrices
X1 = X[:, 0:-1]
X2 = X[:, 1:]

## SVD and rank-2 truncation
r = 2 # rank truncation
[U, S, V] = svd(X1, full_matrices = False)
S = np.diagflat(S) # because numpy returns S as 1D array
Ur = U[:, 0:r] 
Sr = S[0:r, 0:r]
V = np.matrix(V).H
Vr = V[:, 0:r]


### Check dimensions

In [5]:
print(X1.shape, X2.shape)
print(U.shape, S.shape, V.shape)
print(Ur.shape, Sr.shape, Vr.shape)

(400, 199) (400, 199)
(400, 199) (199, 199) (199, 199)
(400, 2) (2, 2) (199, 2)


### Check values

In [7]:
# print(Sr)
# print(Vr)
# print(Ur)


### Build Atilde and DMD Modes 

In [8]:

Atilde = np.dot(np.matrix(Ur).H,np.dot(X2,np.linalg.solve(Sr.T,Vr.T).T))
[D, W] = eig(Atilde)
D = np.diagflat(D)

Phi = np.dot(X2,np.dot(np.linalg.solve(Sr.T,Vr.T).T,W))  # DMD Modes

In [9]:
print(Atilde.shape, W.shape, D.shape, Phi.shape)

# print(Atilde)

# print(D)

# print(W)

# print(Phi)

print(np.matrix(Ur).H.shape)
print(X2.shape)
print(np.linalg.solve(Sr.T,Vr.T).T.shape)
print(Phi.shape)


(2, 2) (2, 2) (2, 2) (400, 2)
(2, 400)
(400, 199)
(199, 2)
(400, 2)


### DMD Spectra

In [10]:

lambda_vec = np.diag(D)
omega = np.log(lambda_vec)/dt

In [11]:

print(lambda_vec, omega)


[0.98440922+0.17589341j 0.98947128+0.14472937j] [-1.04389559e-15+2.8j -1.85703532e-14+2.3j]


In [12]:
import sys
## Compute DMD Solution
x1 = np.array(X[:, 0])
b = np.linalg.lstsq(Phi,x1, rcond = None)[0]
time_dynamics = np.zeros((r,np.size(ti)))

# print(b)

# b*np.exp(omega*ti[2])


In [13]:
for itr in range(np.size(ti)):
    time_dynamics[:,itr] = b*np.exp(omega*ti[itr])
    
X_dmd = np.dot(Phi,time_dynamics)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [16]:
print(X_dmd.shape)

# print(np.iscomplex(X_dmd))

print(Phi.shape, time_dynamics.shape)

print(b.shape)

# print(X_dmd[:,0])

(400, 200)
(400, 2) (2, 200)
(2,)


In [28]:
from matplotlib.colors import LightSource

plt.close('all')

alpha_val = 1.0
colormap = 'gray_r'

fig = plt.figure(figsize = (10,10))
ax_f1 = fig.add_subplot(221, projection='3d')
ax_f2 = fig.add_subplot(222, projection='3d')
ax_f = fig.add_subplot(223, projection='3d')

ls = LightSource(270, 45)

ax_f1.plot_surface(xgrid.T, tgrid.T, np.real(f1.T), \
                   antialiased = False, cmap = colormap, \
                   linewidth = 0, alpha = alpha_val)
# ax_f1.view_init(30,-50)
ax_f1.view_init(40,-120)
ax_f1.set_xlabel(r'$x$')
ax_f1.set_ylabel(r'$t$')
ax_f1.set_zlabel(r'$f_1$')
ax_f1.set_title(r'$f_1(x,t)$')


ax_f2.plot_surface(xgrid.T, tgrid.T, np.real(f2.T), \
                   antialiased = False, cmap = colormap, \
                   linewidth = 0, alpha = alpha_val)
# ax_f2.view_init(30,-50)
ax_f2.view_init(40,-120)
ax_f2.set_xlabel(r'$x$')
ax_f2.set_ylabel(r'$t$')
ax_f2.set_zlabel(r'$f_2$')
ax_f2.set_title(r'$f_2(x,t)$')


ax_f.plot_surface(xgrid.T, tgrid.T, np.real(X), \
                  antialiased = False, cmap = colormap, \
                  linewidth = 0, alpha = alpha_val)
# ax_f.view_init(30,-50)
ax_f.view_init(40,-120)
ax_f.set_xlabel(r'$x$')
ax_f.set_ylabel(r'$t$')
ax_f.set_zlabel(r'$f$')
ax_f.set_title(r'$f(x,t)$')


ax_xdmd = fig.add_subplot(224, projection='3d')

ax_xdmd.plot_surface(xgrid.T, tgrid.T, np.real(X_dmd), \
                  antialiased = False, cmap = colormap, \
                  linewidth = 0, alpha = alpha_val)
ax_xdmd.view_init(40,-120)
# ax_f.view_init(25,-135)
ax_xdmd.set_xlabel(r'$x$')
ax_xdmd.set_ylabel(r'$t$')
ax_xdmd.set_zlabel(r'$f_{\rm DMD}$',rotation = 90)
ax_xdmd.set_title(r'$f_{\rm DMD}(x,t)$')


# plt.savefig('test.pdf', bbox_inches = 'tight')
plt.show()





In [31]:

from matplotlib.colors import LightSource

plt.close('all')

alpha_val = 1.0
colormap = 'gray_r'

fig = plt.figure(figsize = (10,10))
ax_f1 = fig.add_subplot(221, projection='3d')
ax_f2 = fig.add_subplot(222, projection='3d')
ax_f = fig.add_subplot(223, projection='3d')
ax_xdmd = fig.add_subplot(224, projection='3d')

ls = LightSource(270, 45)

ax_f1.plot_surface(xgrid.T, tgrid.T, np.real(f1.T), \
                   antialiased = False, cmap = colormap, \
                   linewidth = 0, alpha = alpha_val)
# ax_f1.view_init(30,-50)
ax_f1.view_init(90, 0)
ax_f1.set_xlabel(r'$x$')
ax_f1.set_ylabel(r'$t$')
ax_f1.set_zlabel(r'$f_1$')
ax_f1.set_title(r'$f_1(x,t)$')


ax_f2.plot_surface(xgrid.T, tgrid.T, np.real(f2.T), \
                   antialiased = False, cmap = colormap, \
                   linewidth = 0, alpha = alpha_val)
# ax_f2.view_init(30,-50)
ax_f2.view_init(90, 0)
ax_f2.set_xlabel(r'$x$')
ax_f2.set_ylabel(r'$t$')
ax_f2.set_zlabel(r'$f_2$')
ax_f2.set_title(r'$f_2(x,t)$')


ax_f.plot_surface(xgrid.T, tgrid.T, np.real(X), \
                  antialiased = False, cmap = colormap, \
                  linewidth = 0, alpha = alpha_val)
# ax_f.view_init(30,-50)
ax_f.view_init(90, 0)
ax_f.set_xlabel(r'$x$')
ax_f.set_ylabel(r'$t$')
ax_f.set_zlabel(r'$f$')
ax_f.set_title(r'$f(x,t)$')



ax_xdmd.plot_surface(xgrid.T, tgrid.T, np.real(X_dmd), \
                  antialiased = False, cmap = colormap, \
                  linewidth = 0, alpha = alpha_val)
ax_xdmd.view_init(90, 0)
# ax_f.view_init(25,-135)
ax_xdmd.set_xlabel(r'$x$')
ax_xdmd.set_ylabel(r'$t$')
ax_xdmd.set_zlabel(r'$f_{\rm DMD}$',rotation = 90)
ax_xdmd.set_title(r'$f_{\rm DMD}(x,t)$')


# plt.savefig('test.pdf', bbox_inches = 'tight')
plt.show()




## Algorithm 1.4

In [ ]:


[U, S, V] = svd(X, full_matrices = False)
S = np.diagflat(S) # because numpy returns S as 1D array);
pc1 = U[:, 0] # first PCA mode
pc2 = U[:, 1] # second PCA mode
time_pc1 = V[:, 0] # temporal evolution of pc1
time_pc2 = V[:, 1] # temporal evolution of pc2



## Algorithm 1.5

In [ ]:
# Requires a fast ICA software package
from sklearn.decomposition import FastICA, PCA

IC = FastICA(n_components=10)
S_ = IC.fit_transform(X)  # Reconstruct signals
ICt = IC.mixing_  # Get estimated mixing matrix

ic1 = IC[0,:] # first ICA mode
ic2 = IC[1, :] # second ICA mode
time_ic1 = ICt[:,0] # temporal evolution of ic1
time_ic2 = ICt[:,1] # temporal evolution of ic2


# [IC, ICt, ~] = fastica(real(X)');
